In [ ]:
from __future__ import print_function
import math
import numpy as np
import random, sys
import matplotlib.pyplot as plt
import _pickle as cPickle

## Source for the dataset : https://www.deepsig.ai/datasets

In [2]:
Xd = cPickle.load(open("Add the PATH to the FILE",'rb'), encoding='latin1')
snrs,mods = map(lambda j: sorted(list(set(map(lambda x: x[j], Xd.keys())))), [1,0])

BPSK = Xd[('BPSK',18)][0:800,:,:]
QPSK = Xd[('QPSK',18)][0:800,:,:]
EPSK = Xd[('8PSK',18)][0:800,:,:]

X = np.vstack((BPSK,QPSK,EPSK))

BPSK_TEST = Xd[('BPSK',18)][800:1000,:,:]
QPSK_TEST = Xd[('QPSK',18)][800:1000,:,:]
EPSK_TEST = Xd[('8PSK',18)][800:1000,:,:]
test = np.vstack((BPSK_TEST,QPSK_TEST,EPSK_TEST))

b = np.zeros(800)
q = np.ones(800)
e = np.empty(800)
e.fill(2)
lbl = np.hstack((b,q,e))

btest = np.zeros(200)
qtest = np.ones(200)
etest = np.empty(200)
etest.fill(2)
lbltest = np.hstack((btest,qtest,etest))

In [3]:
import tensorflow as tf
from tensorflow.keras import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape, Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, BatchNormalization
from tensorflow.keras import backend as K

2022-12-01 21:51:13.555787: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /glob/development-tools/versions/oneapi/2022.3.1/oneapi/vpl/2022.2.5/lib:/glob/development-tools/versions/oneapi/2022.3.1/oneapi/tbb/2021.7.1/env/../lib/intel64/gcc4.8:/glob/development-tools/versions/oneapi/2022.3.1/oneapi/rkcommon/1.10.0/lib:/glob/development-tools/versions/oneapi/2022.3.1/oneapi/ospray_studio/0.11.1/lib:/glob/development-tools/versions/oneapi/2022.3.1/oneapi/ospray/2.10.0/lib:/glob/development-tools/versions/oneapi/2022.3.1/oneapi/openvkl/1.3.1/lib:/glob/development-tools/versions/oneapi/2022.3.1/oneapi/oidn/1.4.3/lib:/glob/development-tools/versions/oneapi/2022.3.1/oneapi/mpi/2021.7.1//libfabric/lib:/glob/development-tools/versions/oneapi/2022.3.1/oneapi/mpi/2021.7.1//lib/release:/glob/development-tools/versions/oneapi/20

In [4]:
batch_size = 128
num_classes = 3

# input image dimensions
img_rows, img_cols = 2, 128

# the data, shuffled and split between train and test sets
x_train = X
y_train = lbl
x_test = test
y_test = lbltest

print(x_train.shape)
print(y_train.shape)

print(x_test.shape)
print(y_test.shape)

(2400, 2, 128)
(2400,)
(600, 2, 128)
(600,)


In [5]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [11]:
print(input_shape)

(2, 128, 1)


In [6]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
#x_train /= 255
#x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (2400, 2, 128, 1)
2400 train samples
600 test samples


In [7]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [ ]:
y_train.shape

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(1, 2),
                 activation='relu',
                 input_shape=input_shape))

dr = 0.5 # dropout rate (%)
model.add(ZeroPadding2D((0, 2)))
model.add(Conv2D(256, (1, 3), padding='valid', activation="relu", name="conv1", kernel_initializer='glorot_uniform'))
model.add(Dropout(dr))
model.add(ZeroPadding2D((0, 2)))
model.add(Conv2D(80, (2, 3), padding="valid", activation="relu", name="conv2", kernel_initializer='glorot_uniform'))
model.add(Dropout(dr))
model.add(Flatten())
model.add(Dense(256, activation='relu', name="dense1"))
model.add(Dropout(dr))
model.add(Dense(num_classes, name="dense2" ))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history=model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=20,
          verbose=1,
          validation_split = 0.1,shuffle=True)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.show()

## Trying out a new model

In [ ]:
model_1 = Sequential()
model_1.add(Conv2D(32, kernel_size=(1, 2),
                 activation='relu',
                 input_shape=input_shape))

dr = 0.25 # dropout rate (%)
model_1.add(ZeroPadding2D((0, 2)))
model_1.add(Conv2D(256, (1, 3), padding='valid', activation="elu", name="conv1"))
model_1.add(Dropout(dr))
model_1.add(ZeroPadding2D((0, 2)))
model_1.add(Conv2D(128, (2, 3), padding="valid", activation="elu", name="conv2"))
model_1.add(BatchNormalization())
model_1.add(Dropout(dr))
model_1.add(Conv2D(32, (1, 2), activation='elu'))
model_1.add(BatchNormalization())
model_1.add(MaxPooling2D(pool_size=(1, 2)))
model_1.add(Dropout(0.25))
model_1.add(Flatten())
model_1.add(Dense(10, activation='relu'))
#model_1.add(Dense(32, activation='relu'))
model_1.add(Dense(num_classes, activation='softmax'))

model_1.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
              metrics=['accuracy'])
model_1.summary()

In [ ]:
history_1=model_1.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=15,
          verbose=1,
          validation_split = 0.1,shuffle=True)

In [8]:
from autoencoder import AutoEncoder

In [15]:
from tensorflow.keras import Model

In [39]:
def create_encoder_model(input_shape):
    model_input = tf.keras.layers.Input(shape=input_shape)

    encoder = Conv2D(32, kernel_size=(1, 2), padding='same', input_shape=input_shape)(model_input)
    encoder=Dropout(0.25)(encoder)
    #encoder = BatchNormalization()(encoder)
    encoder = Activation(activation='relu')(encoder)

    encoder = Conv2D(64, kernel_size=(2, 3), padding='same')(encoder)
    encoder=Dropout(0.25)(encoder)
    #encoder = BatchNormalization()(encoder)
    encoder = Activation(activation='relu')(encoder)
    encoder = MaxPooling2D(pool_size=(1, 3))(encoder)

    encoder = Flatten()(encoder)
    encoder = Dense(128)(encoder)
    #encoder = BatchNormalization()(encoder)
    encoder=Dropout(0.25)(encoder)
    encoder = Activation(activation='relu')(encoder)

    return Model(model_input, encoder)


def create_decoder_model(embedding_shape):
    embedding_a = tf.keras.layers.Input(shape=encoder_model.output_shape)
    decoder = Dense(2*128*1)(embedding_a)
    decoder = Activation(activation='relu')(decoder)
    decoder = Reshape(input_shape)(decoder)

    return Model(embedding_a, decoder)

In [40]:
encoder_model = create_encoder_model(input_shape)

In [35]:
encoder_model.summary()

Model: "model_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 2, 128, 1)]       0         
                                                                 
 conv2d_12 (Conv2D)          (None, 2, 128, 32)        96        
                                                                 
 dropout (Dropout)           (None, 2, 128, 32)        0         
                                                                 
 activation_22 (Activation)  (None, 2, 128, 32)        0         
                                                                 
 conv2d_13 (Conv2D)          (None, 2, 128, 64)        12352     
                                                                 
 dropout_1 (Dropout)         (None, 2, 128, 64)        0         
                                                                 
 activation_23 (Activation)  (None, 2, 128, 64)        0  

In [24]:
encoder_model.output_shape

(None, 128)

In [41]:
decoder_model = create_decoder_model(encoder_model.output_shape)

In [26]:
decoder_model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, None, 128)]       0         
                                                                 
 dense_3 (Dense)             (None, None, 256)         33024     
                                                                 
 activation_9 (Activation)   (None, None, 256)         0         
                                                                 
 reshape (Reshape)           (None, 2, 128, 1)         0         
                                                                 
Total params: 33,024
Trainable params: 33,024
Non-trainable params: 0
_________________________________________________________________


In [42]:
num_classes = 3
epochs = 10

encoder_model = create_encoder_model(input_shape)
decoder_model = create_decoder_model(encoder_model.output_shape)
autoencoder_network = AutoEncoder(encoder_model, decoder_model)
autoencoder_network.compile(loss=tf.keras.losses.categorical_crossentropy, 
                            optimizer=tf.keras.optimizers.Adam(clipvalue=0.1), 
                            metrics=['accuracy'])

In [43]:
autoencoder_network.fit(x_train, x_train,
                        validation_data=(x_test, x_test),
                        batch_size=128,
                        epochs=epochs)

Epoch 1/10
19/19 [==============================] - 1s 45ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/10
19/19 [==============================] - 0s 24ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 3/10
19/19 [==============================] - 0s 24ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 4/10
19/19 [==============================] - 0s 24ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 5/10
19/19 [==============================] - 0s 24ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 6/10
19/19 [==============================] - 0s 24ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 7/10
19/19 [==============================] - 0s 24ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00

In [ ]:
embedding = encoder_model.outputs[-1]

In [ ]:
embedding = Dense(num_classes)(embedding)
embedding = BatchNormalization()(embedding)
embedding = Activation(activation='sigmoid')(embedding)

model = Model(encoder_model.inputs[0], embedding)
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train,
          batch_size=128,
          epochs=epochs,
          validation_data=(x_test, y_test))